## Loading the Datasets

In [1]:
# data analysis
import numpy as np
import pandas as pd

# data visualization
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

print("Setup Complete")

Setup Complete


In [2]:
# loading Data
train_file_path = r"/kaggle/input/titanic/train.csv"
test_file_path = r"/kaggle/input/titanic/test.csv"

train_data = pd.read_csv(train_file_path, index_col='PassengerId')
test_data = pd.read_csv(test_file_path, index_col='PassengerId')

train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Understanding the Data

In [3]:
train_data.info()
# this will show us the columns with null values in them

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [4]:
train_data.describe()
# this gives statistical info about the numerical data

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
train_data.describe(include=['O'])
# same, but for categorical data

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


## Handling Missing Data
(for now we'll just drop all columns with missing data)

In [6]:
# first we'll delete the columns that are irrelevant to us
train_data = train_data.drop(columns=['Cabin','Name','Ticket'])

In [7]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


In [8]:
#and we oughta do the same for the testing dataset for integrity
test_data = test_data.drop(columns=['Cabin','Name','Ticket'])

In [9]:
# NOW we'll replace the missing Ages with the median age..
train_data['Age'] = train_data['Age'].fillna(28)

In [10]:
# let's do the same for testing data..
test_data['Age'] = test_data['Age'].fillna(28)

In [11]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 26.1+ KB


In [12]:
# now we only have missing values in the 'Embarked' column..
# let's replace those missing values with the data's mode
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

In [13]:
# same for testing data except for the median fare this time..
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

## Machine Learning

In [14]:
# first of all, we need to change our categorical data into numerical values for our algorithm
train_data['Sex'] = train_data['Sex'].replace({'male':0,'female':1})
test_data['Sex'] = test_data['Sex'].replace({'male':0,'female':1})

/tmp/ipykernel_13/724041593.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex'] = train_data['Sex'].replace({'male':0,'female':1})
/tmp/ipykernel_13/724041593.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Sex'] = test_data['Sex'].replace({'male':0,'female':1})


In [15]:
train_data['Embarked'] = train_data['Embarked'].replace({'S':0,'C':1,'Q':2})
test_data['Embarked'] = test_data['Embarked'].replace({'S':0,'C':1,'Q':2})

/tmp/ipykernel_13/3368991092.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Embarked'] = train_data['Embarked'].replace({'S':0,'C':1,'Q':2})
/tmp/ipykernel_13/3368991092.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Embarked'] = test_data['Embarked'].replace({'S':0,'C':1,'Q':2})


In [16]:
# First, we'll separate features & target variable
X = train_data.iloc[:,1:]
y = train_data.iloc[:,0]

In [17]:
# Now, we'll split our data into two parts, one for testing and the other for validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y)

In [18]:
# then, we'll define and fit our model, i.e. Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_leaf_nodes=20, random_state=1)
model.fit(X_train, y_train)
predictions = model.predict(X_val)

In [19]:
# we test out various max_leaf_nodes values to figure out a
# range wherein the optimum leaf node value lies..
from sklearn.metrics import accuracy_score
score = accuracy_score(y_val,predictions)
score

0.8430493273542601

In [20]:
# Here we are defining a function to calculate the optimal leaf nodes for our model

def opt_leaves(X_train,y_train,X_val,y_val):
    optimum_leaves=0
    score=0
    for i in range(20,40):
        temp_model = RandomForestClassifier(max_leaf_nodes=i, random_state=1)
        temp_model.fit(X_train, y_train)
        temp_predictions = temp_model.predict(X_val)
        temp_score = accuracy_score(temp_predictions, y_val)
        if temp_score > score:
            score=temp_score
            optimum_leaves=i
    return "Highest Score is {} , where the value max_leaf_nodes is {}".format(score,optimum_leaves)
        
        

In [21]:
opt_leaves(X_train,y_train,X_val,y_val)

'Highest Score is 0.8430493273542601 , where the value max_leaf_nodes is 20'

## Predict the actual test data

In [22]:
model = RandomForestClassifier(max_leaf_nodes=28, random_state=1)
model.fit(X, y)
final_predictions = model.predict(test_data)
final_predictions

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [23]:
output = pd.DataFrame({'PassengerId':test_data.index,'Survived':final_predictions})
output.to_csv('Titanic_submission2', index=False)